In [1]:
#------------------------------------------------------------------------
# WES 207 SBUS Demo Code
#
# This code serves as a baseline for configuring the UART16550 core
# 
#-------------------------------------------------------------------------

from pynq import Overlay
from pynq import MMIO
import time
import numpy as np

print('')
print('done')


done


In [182]:
# Load bitstream
#ol = Overlay('SBUS_DEMO_100M_tx_ext_clks_test5.bit')
ol = Overlay('AXI_UART_SBUS_31.bit')

# Download bitstream to FPGA
t_before_bitstream = time.time()
ol.download()
t_after_bitstream = time.time()
print('  ')
print(t_after_bitstream - t_before_bitstream, 'seconds to program bitstream')

print('')
print('done')

  
0.4186878204345703 seconds to program bitstream

done


In [183]:
# Address space definitions

UART_16550_BASE_ADDRESS        = 0X43C00000
UART_DRIVER_BASE_ADDRESS       = 0X40000000
#UART_DATA_BASE_ADDRESS         = 0X40004000
RC_RECEIVER_BASE_ADDRESS       = 0X43C10000
RC_RECEIVER_DATA_BASE_ADDRESS  = 0X43C14000

#BRAM_BASE_ADDRESS        = 0X42000000

ADDRESS_RANGE1 = 0x1000
ADDRESS_RANGE2 = 0x10000

RX_BUF_REG       =   0x1000        #Receiver Buffer Register              LCR[7] = 0
TX_HOLD_REG      =   0x1000        #Transmitter Holding Register          LCR[7] = 0
INT_EN_REG       =   0x1004        #Interrupt Enable Register             LCR[7] = 0
INT_ID_REG       =   0x1008        #Interrupt Identification Register     LCR[7] = x
FIFO_CNTRL_REG   =   0x1008        #FIFO Control Register                 LCR[7] = x (write), LCR[7] = 1 (read)
LINE_CNTRL_REG   =   0x100C        #Line Control Register                 LCR[7] = x
MODEM_CNTRL_REG  =   0x1010        #Modem Control Register                LCR[7] = x
LINE_STATUS_REG  =   0x1014        #Line Status Register                  LCR[7] = x
MODEM_STATUS_REG =   0x1018        #Modem Status Register                 LCR[7] = x
SCRATCH_REG      =   0x101C        #Scratch Register                      LCR[7] = x
DIV_LATCH_LSB    =   0x1000        #Divisor Latch LSB Register            LCR[7] = 1
DIV_LATCH_MSB    =   0x1004        #Divisor Latch MSB Register            LCR[7] = 1

NUM_BYTES        =   25
START_BYTE     =   0x0F
STOP_BYTE      =   0x00

print('Address Definitions complete')
print('')
print('done')

Address Definitions complete

done


In [184]:
# configuring UART16550 core

UART_core = MMIO(UART_16550_BASE_ADDRESS,ADDRESS_RANGE2, True)
UART_Driver = MMIO(UART_DRIVER_BASE_ADDRESS,ADDRESS_RANGE1, True)
UART_Data = MMIO(UART_DATA_BASE_ADDRESS,ADDRESS_RANGE1, True)
RC_Driver = MMIO(RC_RECEIVER_BASE_ADDRESS,ADDRESS_RANGE1, True)
RC_Data = MMIO(RC_RECEIVER_DATA_BASE_ADDRESS,ADDRESS_RANGE1, True)
#BRAM_Data = MMIO(BRAM_BASE_ADDRESS,ADDRESS_RANGE1, True)


# enable divisor latch, set baud divisor
#UART_core.write(LINE_CNTRL_REG, 0x80)
#UART_core.write(DIV_LATCH_LSB, 0x3F) # divide by 63 for 100MHz AXI clock
#UART_core.write(DIV_LATCH_MSB, 0x00000000)

#UART_core.write(DIV_LATCH_LSB, 0x3E) # divide by 62 for 100MHz AXI clock
#UART_core.write(DIV_LATCH_LSB, 0x00000001) # divide by 1 for external 1.6 MHz BAUD rate clock
#UART_core.write(DIV_LATCH_LSB, 0x00000002) # divide by 2 for external 3.2 MHz BAUD rate clock


# configure data exchange, 100k BAUD, even parity, 2 stop bits, 8 data bits
#UART_core.write(LINE_CNTRL_REG, 0x0000001F)

#UART_core.write(LINE_CNTRL_REG, 0x00000000)
#UART_core.write(LINE_CNTRL_REG, 0x0000000F)

# enable and configure FIFO
#UART_core.write(FIFO_CNTRL_REG, 0x01)

#UART_core.write(FIFO_CNTRL_REG, 0x49)

# reset FIFOs
#UART_core.write(FIFO_CNTRL_REG, 0x07)
#UART_core.write(FIFO_CNTRL_REG, 0x05)
#UART_core.write(LINE_CNTRL_REG, 0x40)

# setting WHO_AM_I value
UART_core.write(SCRATCH_REG,0x45)




print('UART16550 Core Configuration Complete')
print('')
print('done')

MMIO Debug: MMIO(address, size) = (43c00000, 10000 bytes).
MMIO Debug: MMIO(address, size) = (40000000, 1000 bytes).
MMIO Debug: MMIO(address, size) = (40004000, 1000 bytes).
MMIO Debug: MMIO(address, size) = (43c10000, 1000 bytes).
MMIO Debug: MMIO(address, size) = (43c14000, 1000 bytes).
MMIO Debug: Writing 4 bytes to offset 101c: 45
UART16550 Core Configuration Complete

done


In [195]:
# NOTE, starting Receiver core BEFORE Driver core below


# setting ap_start/auto reset on RC Receiver
#RC_Driver.write(0x00,0x01)  # runs once
RC_Driver.write(0x00,0x81) # runs continuously

# setting ap_start/auto reset on UART Driver
#UART_Driver.write(0x00,0x01)  # runs once
UART_Driver.write(0x00,0x81) # runs continuously

# reading back CONFIG registers for HLS cores
CONFIG_REG1 = UART_Driver.read(0x00)
CONFIG_REG2 = RC_Driver.read(0x00)

print('CONFIG_REG UART: ')
print(CONFIG_REG1)
print('')
print('CONFIG_REG RC: ')
print(CONFIG_REG2)
print('')

# this is a basic configuration of the SPI core
# I would read through the docs a bit more and make sure this is correct
# it seems to work as is, 8 bit addres and 8 bit data with SS and CLK

# testing config
#SPI_CORE.write(0x60, 0x006) # enable SPI core in master mode, auto SS
#SPI_CORE.write(0x70, 0xFFFE) # enable SS 0
# reading back config registers
#read1 = UART_core.read(0x60)
#read2 = UART_core.read(0x70)

#print(read1)
#print(read2)
#print('done')

print('')
print('')

MMIO Debug: Writing 4 bytes to offset 0: 81
MMIO Debug: Writing 4 bytes to offset 0: 81
MMIO Debug: Reading 4 bytes from offset 0
MMIO Debug: Reading 4 bytes from offset 0
CONFIG_REG UART: 
131

CONFIG_REG RC: 
131





In [187]:
# testing

test_read = UART_core.read(SCRATCH_REG)

print('WHO_AM_I: ' + str(test_read))

print('')
print('done')

MMIO Debug: Reading 4 bytes from offset 101c
WHO_AM_I: 0

done


In [201]:
# testing reads from UART Driver and RC Reciever Data

index_0 = 0x00
index_1 = 0x04
index_2 = 0x08
index_3 = 0x0C
index_4 = 0x10
index_5 = 0x14
index_6 = 0x18
index_7 = 0x1C 
index_8 = 0x20
index_9 = 0x24
index_10 = 0x28
index_11 = 0x2C
index_12 = 0x30
index_13 = 0x34
index_14 = 0x38
index_15 = 0x3C 
index_16 = 0x40
index_17 = 0x44
index_18 = 0x48
index_19 = 0x4C
index_20 = 0x50
index_21 = 0x54
index_22 = 0x58
index_23 = 0x5C 
index_24 = 0x60

# HLS test read
#TEST_READ = UART_Data.read(index_0)
TEST_READ = RC_Data.read(index_17)

# BRAM test read from HLS core write
#BRAM_READ = BRAM_Data.read(index_0)

# BRAM test write/read
#BRAM_Data.write(0x30,0x45)
#test3 = BRAM_Data.read(0x30)

print('Test Data: ')
print(TEST_READ)
#print('')
#print('BRAM Data: ')
#print(BRAM_READ)
#print('')
#print('Test Write/Read: ')
#print(test3)

print('')
print('done')

MMIO Debug: Reading 4 bytes from offset 44
Test Data: 
224

done


In [110]:

# HLS test write
#UART_Data.write(0x00,0x45)
RC_Data.write(0x00,0x45)

# HLS test read
#TEST_READ = UART_Data.read(0x00)
TEST_READ = RC_Data.read(0x00)

print('Test Data: ')
print(TEST_READ)
print('')

print('')
print('done')

MMIO Debug: Writing 4 bytes to offset 0: 45
MMIO Debug: Reading 4 bytes from offset 0
Test Data: 
69


done


In [ ]:
# allocating storage array for data

ARRAY_SIZE = 50000

buf_data = np.empty([ARRAY_SIZE,], dtype=np.int8)

print('Array allocation complete')
print('')
print('done')

In [ ]:
# test code

# writing to TX register
#UART_core.write(TX_HOLD_REG,0x55)
#UART_core.write(TX_HOLD_REG,0xad)
#UART_core.write(TX_HOLD_REG,0xbe)
#UART_core.write(TX_HOLD_REG,0xef)

print('transmit complete')
print('')
print('done')

In [ ]:
# grabbing data from the UART bus

NUM_BYTES = 25

BIT_ONE = 0x00000001
buffer = [0]*(NUM_BYTES)
BYTES_READ = 0


# resetting RX FIFO
#UART_core.write(FIFO_CNTRL_REG, 0x7)

while(BYTES_READ != (NUM_BYTES)):
    
    #read from the RX FIFO when data present
    while((UART_core.read(LINE_STATUS_REG) & 0x1) != 1):
        pass
    
    buffer[0] = UART_core.read(RX_BUF_REG)
    if ( buffer[0] == START_BYTE):
    #if ( buffer[0]):
        BYTES_READ += 1
        for i in range(NUM_BYTES - 1):
            while((UART_core.read(LINE_STATUS_REG) & 0x1) != 1):
                pass
            buffer[BYTES_READ] = UART_core.read(RX_BUF_REG)
            BYTES_READ += 1
        #if( buffer[24] != STOP_BYTE ):
            #BYTES_READ = 0
    else:
        pass
    #print('Total bytes read: ', BYTES_READ)

    

    
#print(buffer)  # printing buffer contents in decimal
print('')
print(''.join('%02x'%i for i in buffer))   # printing buffer contents in hex

print('')
print('done')

In [ ]:
# testing

WHO_AM_I = UART_core.read(SCRATCH_REG)

print('WHO_AM_I: ' + str(WHO_AM_I))

print('')
print('done')

In [ ]:
# Write OFDM Receiver results to output file for easier comparison
with open('output_compare.dat', 'w') as file_out:
    file_out.write("-----------------------------------------------------------------\n")
    file_out.write("             Golden Output versus OFDM Receiver Output           \n")
    file_out.write("-----------------------------------------------------------------\n\n")
    for k in range(FSIZE):
        file_out.write("index "+str(k)+",\t R: "+str(inp_R[k])+",\t I: "+str(inp_I[k])+",\t Gold: "+str(golden_D[k])+",\t OFDM: "+str(out_D[k])+"\n")

In [ ]:
# parsing buffered packet into channel data

# channel 1   Throttle
# channel 2   Roll
# channel 3   Pitch
# channel 4   yaw

NUM_CHANNELS = 4

channel = [0]*NUM_CHANNELS

# bit masking to parse 11 bit channel data from buffer
channel[0] = (buffer[1]      | buffer[2] << 8)                    & 0x7FF
channel[1] = (buffer[2] >> 3 | buffer[3] << 5)                    & 0x7FF
channel[2] = (buffer[3] >> 6 | buffer[4] << 2 | buffer[5] << 10)  & 0x7FF
channel[3] = (buffer[5] >> 1 | buffer[6] << 7)                    & 0x7FF


print('Throttle Command:  ', channel[0])
print('Roll Command:      ', channel[1])
print('Pitch Command:     ', channel[2])
print('Yaw Command:       ', channel[3])

print('')
print('done')

In [ ]:
#########################################
# old code
#########################################

# grabbing data from the UART bus

BIT_ONE = 0x00000001
buffer = [0]*(NUM_BYTES)
#buffer = [0]*(NUM_BYTES-1)
BYTES_READ = 0

test = [0]*(NUM_BYTES)


while(BYTES_READ != (NUM_BYTES)):
#while(BYTES_READ != (NUM_BYTES - 1)):
    
    #read from the RX FIFO when data present
    #while((mmio.read(LINE_STATUS_REG) & BIT_ONE) != 1):
        #pass
    
    #print(mmio.read(RX_BUF_REG))
    #print(idx)
    
    #if(mmio.read(RX_BUF_REG) != 240):
        #pass
        #print('hard pass')
        
        
    for index in range(NUM_BYTES):
        buffer[index] = mmio.read(RX_BUF_REG)
        BYTES_READ += 1
        
    #temp = mmio.read(RX_BUF_REG)
    #if (temp != 240):
    #    print(temp)
    #else:
    #    buffer[BYTES_READ] = mmio.read(RX_BUF_REG)
    #    BYTES_READ += 1

print(buffer) 
print('')
print(''.join('%02x'%i for i in buffer))
print('')

for i in range(NUM_BYTES):
    test[i] = ~buffer[i]
    
print(test)
print('')
print(''.join('%02x'%i for i in test))

In [ ]:
# testing grabbing channel 1

print('buffer[1]:             ', buffer[1])
print('buffer[2]:             ', buffer[2])
print('buffer[2] last 3 bits: ', buffer[2] & 0x07)

temp = buffer[1] | (buffer[2] << 8)
temp2 = temp & 0x07FF
temp3 = (buffer[1] | buffer[2] << 8) & 0x07FF
 
print('temp:                  ', temp)
print('temp2:                 ', temp2)
print('temp3:                 ', temp3)

start = buffer[0]
print('start byte:            ', start)
channel_1 = (buffer[1] | ( buffer[2] << 8 )) & 0x07FF

print('channel 1:             ', channel_1)

In [ ]:
# TRAVIS IDEAS for infinite graph display

while !quit:
  get packet
  update the graph
  send stuff if the user wants to send stuff
    i.e. break commands